In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install transformers datasets

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from datasets import (Dataset, DatasetDict)
from transformers import (AutoTokenizer,
        AutoModel,
        DataCollatorWithPadding,
        Trainer,
        TrainingArguments,
        AutoConfig)
from sklearn.model_selection import train_test_split

In [ ]:
# Path
my_PATH = "/content/drive/MyDrive/ybigta/Project_1"
os.chdir(my_PATH)
# show file list in the current directory
!dir

In [ ]:
#For loading data
def load_data(ch, t_size, v_size):
  df = pd.read_csv(f"{ch}.csv", usecols= ['Title', 'label'], index_col=0).dropna()
  train_df, test_df = train_test_split(df, test_size = t_size)
  val_df, test_df = train_test_split(test_df, test_size=v_size)

  raw_ds = DatasetDict({"train": Dataset.from_pandas(train_df),
       "validation": Dataset.from_pandas(val_df),
       "test": Dataset.from_pandas(test_df)})
  
  return raw_ds

In [ ]:
ch = "LabeledData_added"
t_size = 0.2
v_size = 0.5

In [ ]:
raw_ds = load_data(ch, t_size, v_size)
raw_ds

In [ ]:
#getting the tokenizer
checkpoint = "klue/roberta-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
#tokenizing data
def tokenize_function(batch):
  return tokenizer(batch['Title'], truncation = True, max_length= 512)

tokenized =raw_ds.map(tokenize_function, batched = True)
tokenized

In [ ]:
#set format of tokenized data for pytorch and get data collator
tokenized.set_format("torch", columns = ["input_ids", "attention_mask", "label"])
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
class LanguageModel(nn.Module):
  def __init__(self, checkpoint):
    super(LanguageModel, self).__init__()

    # Load model with given checkpoint and get the body
    self.transformer = AutoModel.from_pretrained(checkpoint,
                                                   config = AutoConfig.from_pretrained(
                                                       checkpoint,
                                                       output_attentions = True, 
                                                       output_hidden_states = True))
    self.bidir_LSTM = nn.LSTM(768, 50, bidirectional= True) 
    self.flatten = nn.Flatten() 
    self.dense_50 = nn.Linear(100, 50)
  def forward(self, input_ids=None, attention_mask=None,labels=None):
    #Extract outputs from the body
    outputs = self.transformer(input_ids=input_ids, attention_mask=attention_mask)
    #Add custom layers
    LSTM_out = self.bidir_LSTM(outputs.last_hidden_states)
    max_pool_out = torch.max(LSTM_out,1)
    output = F.relu(self.dense_50(max_pool_out))
    return output

  

In [ ]:
model = LanguageModel(checkpoint = checkpoint)

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_dataset["train"], shuffle=True, batch_size = 32, collate_fn = data_collator
)
eval_dataloader = DataLoader(
    tokenized_dataset["valid"], batch_size=32, collate_fn = data_collator
)

In [ ]:
from transformers import AdamW, get_scheduler

optimizer = AdamW(model.parameters(), lr = 5e-5,eps = 1e-8)

num_epoch = 3
num_training_steps = num_epoch * len(train_dataloader)
lr_scheduler = get_scheduler(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps = num_training_steps)

print(num_training_steps)

In [ ]:
from datasets import load_metric
metric = load_metric(["accuracy", "f1"])

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('device:', device)

In [ ]:
from tqdm.auto import tqdm

progress_bar_train = tqdm(range(num_training_steps))
progress_bar_eval = tqdm(range(num_epochs*len(eval_dataloader)))

for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        batch = {k:v.to(device) for k, v in batch_items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar_train.update(1)
    model.eval()
    for batch in eval_dataloader:
        batch = {k:v.to(device) for k, v in batch_items()}
        with torch.no.grad():
            outputs = model(**batch)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        metric.add_batch(predictions = predictions, references=batch["label"])
        progress_bar_eval.update(1)
        
    print(metric.compute())

